In [1]:
import requests
import json
import csv
import time
import pandas as pd

In [2]:
# Test connection to API

base = "https://www.balldontlie.io/api/v1/"
url = "games"


seasons = 2018
query = f'seasons[]={seasons}&per_page=100'
params = f'?{query}' 

In [3]:
res = requests.get(base+url)

In [4]:
res.status_code

200

In [5]:
res.json()['meta']

{'total_pages': 2051,
 'current_page': 1,
 'next_page': 2,
 'per_page': 25,
 'total_count': 51255}

In [6]:
# Function to retrieve data by season

def get_season_data(stat, seasons):
    url = f"https://www.balldontlie.io/api/v1/{stat}?seasons[]={season}&per_page=100"
    res = requests.get(url)
    status = res.status_code
    print(f"Status: {status}")

    # Get total number of pages
    pages = res.json()['meta']['total_pages']
    filename = f'C://Users/brian/lighthouse-data-notes/Final_Project/{stat}_{season}_season.csv'

    # Creates a Dataframe with the flattened JSON results
    df = pd.json_normalize(res.json()['data'])

    # Repeat get request for each page from page 2 onwards, adding it to the dataframe
    for page in range(2,pages+1):
        url = f"https://www.balldontlie.io/api/v1/{stat}?seasons[]={season}&per_page=100&page={page}"
        res = requests.get(url)
        status = res.status_code

        # If we get anything but 200 as status, tell us what page
        if status != 200:
            print(f'Page {page} Status: {status}')

        # Normalize JSON result and add results to dataframe
        df_page = pd.json_normalize(res.json()['data'])
        df = pd.concat([df, df_page])

        # Sleep for 1 second after each call. BDL API is limited to 60 per minute
        time.sleep(1)

    # Save dataframe to csv
    df.to_csv(filename, index=False)
    print(f'{filename} saved')

In [10]:
# Get Games Data

seasons = [2019,2020,2021,2022]
# seasons = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
# seasons = [1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]
for season in seasons:
    print(f'Getting game data for {season} season')
    get_season_data('games', season)

Getting game data for 2019 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/games_2019_season.csv saved
Getting game data for 2020 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/games_2020_season.csv saved
Getting game data for 2021 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/games_2021_season.csv saved
Getting game data for 2022 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/games_2022_season.csv saved


In [11]:
# Get Player stats

for season in seasons:
    print(f'Getting player stats for {season} season')
    get_season_data('stats', season)

Getting player stats for 2019 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/stats_2019_season.csv saved
Getting player stats for 2020 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/stats_2020_season.csv saved
Getting player stats for 2021 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/stats_2021_season.csv saved
Getting player stats for 2022 season
Status: 200
C://Users/brian/lighthouse-data-notes/Final_Project/stats_2022_season.csv saved
